## Export data from the old Database, and import processed data in the new DB

In [1]:
# imports
import psycopg2
import numpy as np
import os
import pandas as pd
from dotenv import load_dotenv

In [75]:
load_dotenv()

DB_NAME_OLD = os.getenv("DB_NAME_OLD")
DB_NAME_NEW = os.getenv("DB_NAME_NEW")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
PORT = os.getenv("PORT")
DEV = os.getenv("DEV")

# Connect to the old database
connection_old = psycopg2.connect(user="DB_USER",
                              password="DB_PASSWORD",
                              host="DEV",
                              port="PORT",
                              database="DB_NAME_OLD")

# Connect to the new database
connection_new = psycopg2.connect(user="DB_USER",
                              password="DB_PASSWORD",
                              host="DEV",
                              port="PORT",
                              database="DB_NAME_NEW")

### User ratings

In [136]:
# Data from user_groa_ratings
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date, a.rating
        FROM user_groa_ratings a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_groa_ratings = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
rating = []

for row in user_groa_ratings:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2])
    date.append(row[3])
    rating.append(row[4])
    
user_groa_ratings = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'rating': rating
})
print(user_groa_ratings.shape)
user_groa_ratings.head()

(9492, 5)


,id,user_id,movie_id,date,rating
0,9234,312517,1051904,2016-05-23,2.0
1,7152,311074,1051904,2016-05-23,2.0
2,8853,312517,1057500,2013-02-11,2.5
3,6773,311074,1057500,2013-02-11,2.5
4,9282,312517,1059786,2017-02-16,3.0


In [138]:
# Check for duplicates
pd.concat(g for _, g in user_groa_ratings.groupby(['id']) if len(g) > 1)

,id,user_id,movie_id,date,rating
787,549,42593,1895587,2019-03-15,1.5
4414,549,42593,7785302,2019-03-15,1.5
786,1054,72527,1895587,2019-03-15,1.5
4413,1054,72527,7785302,2019-03-15,1.5
785,1118,70254,1895587,2019-03-15,1.5
...,...,...,...,...,...
8095,15624,316396,1014759,2018-12-18,3.0
757,15965,316396,1895587,2019-03-15,1.5
4384,15965,316396,7785302,2019-03-15,1.5
4559,16107,315729,1799508,2020-04-09,4.0


In [139]:
# drop ALL duplicate values 
user_groa_ratings.drop_duplicates(subset =['id'], keep = False, inplace = True) 

In [140]:
# Drop the id
user_groa_ratings = user_groa_ratings.drop(['id'], axis=1)
user_groa_ratings.head()

,user_id,movie_id,date,rating
0,312517,1051904,2016-05-23,2.0
1,311074,1051904,2016-05-23,2.0
2,312517,1057500,2013-02-11,2.5
3,311074,1057500,2013-02-11,2.5
4,312517,1059786,2017-02-16,3.0


In [141]:
# Add the source
user_groa_ratings['source'] = 'groa'
user_groa_ratings.head()

,user_id,movie_id,date,rating,source
0,312517,1051904,2016-05-23,2.0,groa
1,311074,1051904,2016-05-23,2.0,groa
2,312517,1057500,2013-02-11,2.5,groa
3,311074,1057500,2013-02-11,2.5,groa
4,312517,1059786,2017-02-16,3.0,groa


In [142]:
# Data from user_imdb_ratings
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date, a.rating
        FROM user_imdb_ratings a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_imdb_ratings = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
rating = []

for row in user_imdb_ratings:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2])
    date.append(row[3])
    rating.append(row[4])
    
user_imdb_ratings = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'rating': rating
})
print(user_imdb_ratings.shape)
user_imdb_ratings.head()

(18498, 5)


,id,user_id,movie_id,date,rating
0,19927,11,1396484,2017-12-31,4.5
1,20312,11,1119646,2018-12-18,4.5
2,20157,11,1119646,2018-12-18,4.5
3,19969,11,1119646,2018-12-18,4.5
4,19634,11,1119646,2018-12-18,4.5


In [143]:
# Check for duplicates
pd.concat(g for _, g in user_imdb_ratings.groupby(['id']) if len(g) > 1)

,id,user_id,movie_id,date,rating
6299,57,11,0229208,2017-02-16,4.5
10702,57,11,0086879,2017-02-16,4.5
6298,291,11,0229208,2017-02-15,4.5
10701,291,11,0086879,2017-02-15,4.5
6297,832,11,0229208,2017-02-15,4.5
...,...,...,...,...,...
10620,19533,11,0086879,2017-02-16,4.5
6216,19586,11,0229208,2017-02-16,4.5
10619,19586,11,0086879,2017-02-16,4.5
6215,19900,11,0229208,2017-02-16,4.5


In [144]:
# drop ALL duplicate values 
user_imdb_ratings.drop_duplicates(subset =['id'], keep = False, inplace = True) 

In [145]:
# Drop the id
user_imdb_ratings = user_imdb_ratings.drop(['id'], axis=1)

# Add the source
user_imdb_ratings['source'] = 'imdb'
user_imdb_ratings.head()

,user_id,movie_id,date,rating,source
0,11,1396484,2017-12-31,4.5,imdb
1,11,1119646,2018-12-18,4.5,imdb
2,11,1119646,2018-12-18,4.5,imdb
3,11,1119646,2018-12-18,4.5,imdb
4,11,1119646,2018-12-18,4.5,imdb


In [146]:
# Data from user_letterboxd_ratings
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date, a.rating
        FROM user_letterboxd_ratings a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_letterboxd_ratings = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
rating = []

for row in user_letterboxd_ratings:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2])
    date.append(row[3])
    rating.append(row[4])
    
user_letterboxd_ratings = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'rating': rating
})
print(user_letterboxd_ratings.shape)
user_letterboxd_ratings.head()

(108075, 5)


,id,user_id,movie_id,date,rating
0,15204,15061,0462499,2012-10-18,3.0
1,15093,15061,0114369,2012-09-21,4.0
2,15131,15061,0056869,2012-09-21,3.5
3,8000,7968,0114369,2012-09-20,4.0
4,8047,7968,0372784,2012-09-20,3.5


In [147]:
# Check for duplicates
pd.concat(g for _, g in user_letterboxd_ratings.groupby(['id']) if len(g) > 1)

,id,user_id,movie_id,date,rating
81935,30,1,0086879,2020-02-13,3.5
81936,30,1,0229208,2020-02-13,3.5
80086,5023,1,0086879,2020-02-14,4.5
80093,5023,1,0229208,2020-02-14,4.5
68813,5479,5028,2651158,2013-12-14,4.0
...,...,...,...,...,...
81843,310396,310305,0229208,2020-02-14,3.5
93743,310579,48484,0086879,2020-02-14,3.5
93744,310579,48484,0229208,2020-02-14,3.5
81661,310772,310672,0086879,2020-02-14,3.5


In [148]:
# drop ALL duplicate values 
user_letterboxd_ratings.drop_duplicates(subset =['id'], keep = False, inplace = True) 

In [149]:
# Drop the id
user_letterboxd_ratings = user_letterboxd_ratings.drop(['id'], axis=1)

# Add the source
user_letterboxd_ratings['source'] = 'letterboxd'
user_letterboxd_ratings.head()

,user_id,movie_id,date,rating,source
0,15061,0462499,2012-10-18,3.0,letterboxd
1,15061,0114369,2012-09-21,4.0,letterboxd
2,15061,0056869,2012-09-21,3.5,letterboxd
3,7968,0114369,2012-09-20,4.0,letterboxd
4,7968,0372784,2012-09-20,3.5,letterboxd


In [150]:
# Shapes of ratings tables
print('shape of user_groa_ratings:', user_groa_ratings.shape)
print('shape of user_imdb_ratings:', user_imdb_ratings.shape)
print('shape of user_letterboxd_ratings:', user_letterboxd_ratings.shape)
print('Total rows:', user_groa_ratings.shape[0] + user_imdb_ratings.shape[0] + user_letterboxd_ratings.shape[0])

shape of user_groa_ratings: (9374, 5)
shape of user_imdb_ratings: (18328, 5)
shape of user_letterboxd_ratings: (106881, 5)
Total rows: 134583


In [151]:
# Concatenate the above three dataframes
frames = [user_groa_ratings, user_imdb_ratings, user_letterboxd_ratings]
user_ratings = pd.concat(frames)
print(user_ratings.shape)
user_ratings.head()

(134583, 5)


,user_id,movie_id,date,rating,source
0,312517,1051904,2016-05-23,2.0,groa
1,311074,1051904,2016-05-23,2.0,groa
2,312517,1057500,2013-02-11,2.5,groa
3,311074,1057500,2013-02-11,2.5,groa
4,312517,1059786,2017-02-16,3.0,groa


In [152]:
# Save the dataframe into csv file
user_ratings.to_csv('user_ratings.csv', index=False)

### User reviews

In [182]:
# Data from user_groa_reviews
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date, a.review, a.tags
        FROM user_groa_reviews a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_groa_reviews = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
review_title = []
review_text = []
tags = []
source = []

for row in user_groa_reviews:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2]),
    date.append(row[3])
    review_title.append(np.NaN)
    review_text.append(row[4])
    tags.append(row[5])
    source.append('groa')

user_groa_reviews = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'review_title': review_title,
    'review_text': review_text,
    'tags': tags,
    'source': source
})
print(user_groa_reviews.shape)
user_groa_reviews.head()

(315, 8)


,id,user_id,movie_id,date,review_title,review_text,tags,source
0,9519,312517,1170358,2013-12-30,NaN,I curled my upper lip so hard that it creased....,,groa
1,7433,311074,1170358,2013-12-30,NaN,I curled my upper lip so hard that it creased....,,groa
2,7364,311074,1250777,2012-09-21,NaN,"A marvel of filmmaking fun, fists, frisson, an...",,groa
3,7398,311074,1228987,2012-11-24,NaN,Basically superfluous. The original was perfec...,,groa
4,9484,312517,1228987,2012-11-24,NaN,Basically superfluous. The original was perfec...,,groa


In [183]:
# Check for duplicates
pd.concat(g for _, g in user_groa_reviews.groupby(['id']) if len(g) > 1)

,id,user_id,movie_id,date,review_title,review_text,tags,source
36,7440,311074,1935179,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,groa
146,7440,311074,2514592,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,groa
35,9525,312517,1935179,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,groa
129,9525,312517,2514592,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,groa


In [184]:
# drop ALL duplicate values 
user_groa_reviews.drop_duplicates(subset =['id'], keep = False, inplace = True)

# Drop the id
user_groa_reviews = user_groa_reviews.drop(['id'], axis=1)
user_groa_reviews.head()

,user_id,movie_id,date,review_title,review_text,tags,source
0,312517,1170358,2013-12-30,NaN,I curled my upper lip so hard that it creased....,,groa
1,311074,1170358,2013-12-30,NaN,I curled my upper lip so hard that it creased....,,groa
2,311074,1250777,2012-09-21,NaN,"A marvel of filmmaking fun, fists, frisson, an...",,groa
3,311074,1228987,2012-11-24,NaN,Basically superfluous. The original was perfec...,,groa
4,312517,1228987,2012-11-24,NaN,Basically superfluous. The original was perfec...,,groa


In [185]:
# Data from user_letterboxd_reviews
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date, a.review, a.tags
        FROM user_letterboxd_reviews a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_letterboxd_reviews = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
review_title = []
review_text = []
tags = []
source = []

for row in user_letterboxd_reviews:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2]),
    date.append(row[3])
    review_title.append(np.NaN)
    review_text.append(row[4])
    tags.append(row[5])
    source.append('letterboxd')

user_letterboxd_reviews = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'review_title': review_title,
    'review_text': review_text,
    'tags': tags,
    'source': source
})
print(user_letterboxd_reviews.shape)
user_letterboxd_reviews.head()

(11210, 8)


,id,user_id,movie_id,date,review_title,review_text,tags,source
0,593,19614,0033870,2012-09-21,NaN,Humphrey Bogart makes this film with the colde...,,letterboxd
1,11375,276525,0030287,2019-02-14,NaN,"""I like my convictions undiluted -- same as I ...",,letterboxd
2,12517,306631,0030287,2019-02-14,NaN,"""I like my convictions undiluted -- same as I ...",,letterboxd
3,12164,297984,0030287,2019-02-14,NaN,"""I like my convictions undiluted -- same as I ...",,letterboxd
4,8133,197098,0030287,2019-02-14,NaN,"""I like my convictions undiluted -- same as I ...",,letterboxd


In [186]:
# Check for duplicates
pd.concat(g for _, g in user_letterboxd_reviews.groupby(['id']) if len(g) > 1)

,id,user_id,movie_id,date,review_title,review_text,tags,source
7483,91,1,2514592,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",None,letterboxd
8498,91,1,1935179,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",None,letterboxd
7537,187,5028,2514592,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,letterboxd
8552,187,5028,1935179,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,letterboxd
7536,284,5028,2514592,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,letterboxd
...,...,...,...,...,...,...,...,...
8484,12080,294333,1935179,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,letterboxd
7466,12338,301078,2514592,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,letterboxd
8481,12338,301078,1935179,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,letterboxd
7436,12634,48485,2514592,2018-01-11,NaN,"""The deal was for the gun, not for bullets."" ""...",,letterboxd


In [187]:
# drop ALL duplicate values 
user_letterboxd_reviews.drop_duplicates(subset =['id'], keep = False, inplace = True)

# Drop the id
user_letterboxd_reviews = user_letterboxd_reviews.drop(['id'], axis=1)
user_letterboxd_reviews.head()

,user_id,movie_id,date,review_title,review_text,tags,source
0,19614,0033870,2012-09-21,NaN,Humphrey Bogart makes this film with the colde...,,letterboxd
1,276525,0030287,2019-02-14,NaN,"""I like my convictions undiluted -- same as I ...",,letterboxd
2,306631,0030287,2019-02-14,NaN,"""I like my convictions undiluted -- same as I ...",,letterboxd
3,297984,0030287,2019-02-14,NaN,"""I like my convictions undiluted -- same as I ...",,letterboxd
4,197098,0030287,2019-02-14,NaN,"""I like my convictions undiluted -- same as I ...",,letterboxd


In [188]:
# Shapes of user review tables
print('shape of user_groa_reviews:', user_groa_reviews.shape)
print('shape of user_letterboxd_reviews:', user_letterboxd_reviews.shape)
print('Total rows:', user_groa_reviews.shape[0] + user_letterboxd_reviews.shape[0])

shape of user_groa_reviews: (311, 7)
shape of user_letterboxd_reviews: (10996, 7)
Total rows: 11307


In [189]:
# Concatenate the above dataframes
frames = [user_groa_reviews, user_letterboxd_reviews]
user_reviews = pd.concat(frames)
print(user_reviews.shape)
user_reviews.head()

(11307, 7)


,user_id,movie_id,date,review_title,review_text,tags,source
0,312517,1170358,2013-12-30,NaN,I curled my upper lip so hard that it creased....,,groa
1,311074,1170358,2013-12-30,NaN,I curled my upper lip so hard that it creased....,,groa
2,311074,1250777,2012-09-21,NaN,"A marvel of filmmaking fun, fists, frisson, an...",,groa
3,311074,1228987,2012-11-24,NaN,Basically superfluous. The original was perfec...,,groa
4,312517,1228987,2012-11-24,NaN,Basically superfluous. The original was perfec...,,groa


In [190]:
# Save the dataframe into csv file
user_reviews.to_csv('user_reviews.csv', index=False)

### User watched

In [191]:
# Data from user_letterboxd_watched
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date
        FROM user_letterboxd_watched a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_letterboxd_watched = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
source = []

for row in user_letterboxd_watched:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2]),
    date.append(row[3])
    source.append('letterboxd')

user_letterboxd_watched = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'source': source
})
print(user_letterboxd_watched.shape)
user_letterboxd_watched.head()

(159231, 5)


,id,user_id,movie_id,date,source
0,1177,296,0371746,2012-09-20,letterboxd
1,1227,296,0033467,2012-09-20,letterboxd
2,1261,296,0114814,2012-09-20,letterboxd
3,1764,296,0443536,2012-11-05,letterboxd
4,1785,296,0358082,2012-11-09,letterboxd


In [192]:
# Check for duplicates
pd.concat(g for _, g in user_letterboxd_watched.groupby(['id']) if len(g) > 1)

,id,user_id,movie_id,date,source
113951,36,1,0229208,2020-02-13,letterboxd
113952,36,1,0086879,2020-02-13,letterboxd
94814,1181,296,1655442,2012-09-20,letterboxd
94816,1181,296,1825978,2012-09-20,letterboxd
100866,1344,296,0229208,2012-09-20,letterboxd
...,...,...,...,...,...
113693,316679,316396,2514592,2018-12-18,letterboxd
156218,316688,316396,1014759,2018-12-18,letterboxd
156219,316688,316396,2049386,2018-12-18,letterboxd
55389,317020,316396,1895587,2019-03-15,letterboxd


In [193]:
# drop ALL duplicate values 
user_letterboxd_watched.drop_duplicates(subset =['id'], keep = False, inplace = True)

# Drop the id
user_letterboxd_watched = user_letterboxd_watched.drop(['id'], axis=1)
user_letterboxd_watched.head()

,user_id,movie_id,date,source
0,296,0371746,2012-09-20,letterboxd
1,296,0033467,2012-09-20,letterboxd
2,296,0114814,2012-09-20,letterboxd
3,296,0443536,2012-11-05,letterboxd
4,296,0358082,2012-11-09,letterboxd


In [194]:
# Save the dataframe into csv file
user_letterboxd_watched.to_csv('user_watched.csv', index=False)

### User Watchlist

In [32]:
# Data from user_letterboxd_watchlist
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date
        FROM user_letterboxd_watchlist a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_letterboxd_watchlist = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
source = []

for row in user_letterboxd_watchlist:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2]),
    date.append(row[3])
    source.append('letterboxd')

user_letterboxd_watchlist = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'source': source
})

# drop ALL duplicate values 
user_letterboxd_watchlist.drop_duplicates(subset =['id'], keep = False, inplace = True)

# Drop the id
user_letterboxd_watchlist = user_letterboxd_watchlist.drop(['id'], axis=1)
user_letterboxd_watchlist.head()

,user_id,movie_id,date,source
0,NaN,0114746,2012-09-20,letterboxd
1,NaN,0388795,2012-09-20,letterboxd
2,NaN,0408306,2012-09-20,letterboxd
3,NaN,0057129,2012-11-20,letterboxd
4,NaN,0114279,2012-11-20,letterboxd


In [199]:
# How many missing values?
print(user_letterboxd_watchlist.shape)
user_letterboxd_watchlist.isnull().sum()

(92549, 4)


user_id     2086
movie_id       0
date           0
source         0
dtype: int64

In [207]:
# Drop values with missing data
user_letterboxd_watchlist = user_letterboxd_watchlist.dropna()

# Convert user_id from float into integer
user_letterboxd_watchlist['user_id'] = user_letterboxd_watchlist['user_id'].astype(int)
user_letterboxd_watchlist.shape

(90463, 4)

In [202]:
# Data from user_groa_watchlist
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date
        FROM user_groa_watchlist a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_groa_watchlist = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
source = []

for row in user_groa_watchlist:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2])
    date.append(row[3])
    source.append('groa')

user_groa_watchlist = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'source': source
})

# drop ALL duplicate values 
user_groa_watchlist.drop_duplicates(subset =['id'], keep = False, inplace = True)

# Drop the id
user_groa_watchlist = user_groa_watchlist.drop(['id'], axis=1)
user_groa_watchlist.head()

,user_id,movie_id,date,source
0,312517,1093842,2013-07-18,groa
1,312517,1091722,2013-07-17,groa
2,311074,1091722,2013-07-17,groa
3,312517,1069238,2012-10-23,groa
4,311074,1069238,2012-10-23,groa


In [208]:
# Shapes of user review tables
print('shape of user_groa_watchlist:', user_groa_watchlist.shape)
print('shape of user_letterboxd_watchlist:', user_letterboxd_watchlist.shape)
print('Total rows:', user_groa_watchlist.shape[0] + user_letterboxd_watchlist.shape[0])

# Concatenate the dataframes
frames = [user_groa_watchlist, user_letterboxd_watchlist]
user_watchlist = pd.concat(frames)
print(user_watchlist.shape)
user_watchlist.head()

shape of user_groa_watchlist: (2627, 4)
shape of user_letterboxd_watchlist: (90463, 4)
Total rows: 93090
(93090, 4)


,user_id,movie_id,date,source
0,312517,1093842,2013-07-18,groa
1,312517,1091722,2013-07-17,groa
2,311074,1091722,2013-07-17,groa
3,312517,1069238,2012-10-23,groa
4,311074,1069238,2012-10-23,groa


In [209]:
# Save the dataframe into csv file
user_watchlist.to_csv('user_watchlist.csv', index=False)

### Movie reviews

In [3]:
# Data from user_imdb_ratings
cursor_old = connection_old.cursor()
query = """SELECT * FROM imdb_reviews;"""
cursor_old.execute(query)
movie_reviews = cursor_old.fetchall()
cursor_old.close()

movie_id = []
review_date = []
user_rating = []
helpful_num = []
helpful_denom = []
user_name = []
review_text = []
review_title = []

for row in movie_reviews:
    movie_id.append(row[0])
    review_date.append(row[1])
    user_rating.append(row[2])
    helpful_num.append(row[3])
    helpful_denom.append(row[4])
    user_name.append(row[5])
    review_text.append(row[6])
    review_title.append(row[7])
    
movie_reviews = pd.DataFrame({
    'movie_id': movie_id,
    'review_date': review_date,
    'user_rating': user_rating,
    'helpful_num': helpful_num,
    'helpful_denom': helpful_denom,
    'user_name': user_name,
    'review_text': review_text,
    'review_title': review_title
})
print(movie_reviews.shape)
movie_reviews.head()

(3462445, 8)


,movie_id,review_date,user_rating,helpful_num,helpful_denom,user_name,review_text,review_title
0,0119548,2003-05-18,8.0,10,10,dwpollar,1st watched 5/18/2003 - 8 out of 10(Dir-Robert...,Great little movie with brutal relationship h...
1,0119548,2001-04-24,6.0,8,9,=G=,Little City refers to San Francisco of which m...,A fun rest stop for mature channel surfers.\n
2,0119548,2001-09-04,9.0,7,8,karier13,This was not a fast paced movie and as some sa...,honest movie showing real feelings\n
3,0119548,1999-04-27,8.0,6,7,Dawn-31,Im a big Jon Bon Jovi fan so the only reason I...,It was actually a good movie!\n
4,0119548,2005-01-31,8.0,2,2,cinfante-1,I thoroughly enjoyed the movie for the reasons...,"San Francisco, modern singles angst, super ac..."


In [ ]:
# tbc